# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the MLP2024_25_CW1_Spec.pdf (see Learn,  Assignment Submission, Coursework 1). The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for num_epochs epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_xlabel('Accuracy')
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [8]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
import sys
import os 
if os.name == 'nt':
    os.environ['MLP_DATA_DIR'] = 'C:\\Schoolwork\\4\\MLP\\mlpractical\\data'
    sys.path.append('C:\\Schoolwork\\4\\MLP')
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102019 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

KeysView(NpzFile '/afs/inf.ed.ac.uk/user/s22/s2212195/mlpractical/data/emnist-train.npz' with keys: inputs, targets)
KeysView(NpzFile '/afs/inf.ed.ac.uk/user/s22/s2212195/mlpractical/data/emnist-valid.npz' with keys: inputs, targets)


In [9]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

%pip install tqdm

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule
from mlp.optimisers import Optimiser

# Setup hyperparameters
learning_rate = 0.001
num_epochs = 100
stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 32

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

# Create model with ONE hidden layer
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

Note: you may need to restart the kernel to use updated packages.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 0.6s to complete
    error(train)=1.23e+00, acc(train)=6.56e-01, error(valid)=1.23e+00, acc(valid)=6.55e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 2: 0.6s to complete
    error(train)=1.12e+00, acc(train)=6.84e-01, error(valid)=1.13e+00, acc(valid)=6.77e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
def train_models(
        models, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Run the optimiser for num_epochs epochs (full passes through the training set)
    # printing statistics every epoch.
    statss = []
    keyss = []
    run_times = []
    for model in models:
        optimiser = Optimiser(
            model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)
        stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)
        statss.append(stats)
        keyss.append(keys)
        run_times.append(run_time)
    
    return statss, keyss, run_times

In [18]:
# Train width models

learning_rate = 0.001
num_epochs = 100
stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 32

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model0 = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

hidden_dim = 64

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model1 = MultipleLayerModel([
    AffineLayer(input_dim, 64, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(64, output_dim, weights_init, biases_init) # output layer
])

hidden_dim = 128

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model2 = MultipleLayerModel([
    AffineLayer(input_dim, 128, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(128, output_dim, weights_init, biases_init) # output layer
])

models = [model0,model1,model2]

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
statss, keyss, run_times= train_models(
    models, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 0.6s to complete
    error(train)=1.22e+00, acc(train)=6.58e-01, error(valid)=1.24e+00, acc(valid)=6.54e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 2: 0.6s to complete
    error(train)=1.08e+00, acc(train)=6.93e-01, error(valid)=1.10e+00, acc(valid)=6.84e-01


...

Epoch 98: 1.5s to complete
    error(train)=1.62e-01, acc(train)=9.38e-01, error(valid)=9.33e-01, acc(valid)=8.09e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 99: 1.6s to complete
    error(train)=1.65e-01, acc(train)=9.36e-01, error(valid)=9.55e-01, acc(valid)=8.06e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 100: 1.6s to complete
    error(train)=1.54e-01, acc(train)=9.41e-01, error(valid)=9.45e-01, acc(valid)=8.10e-01


In [16]:
def plot_stats(num_models,statss,keyss,plot_name,labels):
    colors = ['r','b','g']

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for i in range(0,num_models):
        for k in ['error(train)', 'error(valid)']:
            if k == 'error(valid)':
                ax_1.plot(np.arange(1, statss[i].shape[0]) * stats_interval, 
                        statss[i][1:, keyss[i][k]], color=colors[i], label=str(labels[i]+' '+k), linestyle='dashed')
            else:
                ax_1.plot(np.arange(1, statss[i].shape[0]) * stats_interval, 
                        statss[i][1:, keyss[i][k]], color=colors[i], label=str(labels[i]+' '+k))
    ax_1.legend(loc=2)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')
    ax_1.grid('on')
    fig_1.tight_layout()
    fig_1.savefig('error_curve_' + plot_name +'.pdf')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for i in range(0,num_models):
        for k in ['acc(train)', 'acc(valid)']:
            if k == 'acc(valid)':
                ax_2.plot(np.arange(1, statss[i].shape[0]) * stats_interval, 
                        statss[i][1:, keyss[i][k]], color=colors[i],label=str(labels[i]+' '+k), linestyle='dashed')
            else:
                ax_2.plot(np.arange(1, statss[i].shape[0]) * stats_interval, 
                        statss[i][1:, keyss[i][k]], color=colors[i],label=str(labels[i]+' '+k))
    ax_2.legend(loc=2)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_xlabel('Accuracy')
    ax_2.grid('on')
    fig_2.tight_layout()
    fig_2.savefig('acc_curve_' + plot_name +'.pdf')

    return fig_1, ax_1, fig_2, ax_2

In [5]:
def print_stats(stats,keys):
    for i in range(0,3):
        print('Val acc: ' + str(stats[i][-1, keys[i]['acc(valid)']]) + ', Train Error: ' + str(stats[i][-1, keys[i]['error(train)']]) + ', Val Error: ' + str(stats[i][-1, keys[i]['error(valid)']]))


In [19]:
_ = plot_stats(3,statss,keyss,'width',['height 32','height 64','height 128'])
print_stats(statss,keyss)

Val acc: 0.7922784810126582, Train Error: 0.5376225951957342, Val Error: 0.6812726998852612
Val acc: 0.8103797468354424, Train Error: 0.33192823440515584, Val Error: 0.6855164859978709
Val acc: 0.8096835443037972, Train Error: 0.15421267803278843, Val Error: 0.9450342520887643


In [ ]:
# Create model with TWO hidden layers
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # first hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # second hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

In [13]:
# Train depth models

learning_rate = 0.001
num_epochs = 100
stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 128

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model0 = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model1 = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

model2 = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

models = [model0,model1,model2]

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

statss, keyss, run_times= train_models(
    models, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 1.5s to complete
    error(train)=9.06e-01, acc(train)=7.37e-01, error(valid)=9.25e-01, acc(valid)=7.28e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 2: 1.5s to complete
    error(train)=6.99e-01, acc(train)=7.91e-01, error(valid)=7.33e-01, acc(valid)=7.80e-01


...

Epoch 98: 2.0s to complete
    error(train)=1.27e-01, acc(train)=9.52e-01, error(valid)=1.69e+00, acc(valid)=8.21e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 99: 2.0s to complete
    error(train)=1.17e-01, acc(train)=9.55e-01, error(valid)=1.61e+00, acc(valid)=8.23e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 100: 2.0s to complete
    error(train)=1.09e-01, acc(train)=9.58e-01, error(valid)=1.64e+00, acc(valid)=8.24e-01


In [17]:
_ = plot_stats(3,statss,keyss,'depth',['depth 1','depth 2','depth 3'])
print_stats(statss,keyss)

Val acc: 0.8032278481012656, Train Error: 0.16074452172762366, Val Error: 0.9715806449835789
Val acc: 0.8218987341772153, Train Error: 0.09157010820076034, Val Error: 1.512657281322021
Val acc: 0.8244303797468355, Train Error: 0.10910526489429367, Val Error: 1.641518574703702
